In [ ]:
import keras
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Input, Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.utils import get_file
import tarfile
import os
import numpy as np
import json
from collections import Counter
from keras.optimizers import SGD

In [ ]:
path = get_file('pet_images', 'http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz')
tar = tarfile.open(path)
tar.extractall()
tar.close()
os.rename('images', 'pet_images')

In [ ]:
pet_images_fn = [fn for fn in os.listdir('pet_images') if fn.endswith('.jpg')]
labels = []
idx_to_labels = []
label_to_idx = {}
for fn in pet_images_fn:
    label, _ = fn.rsplit('_', 1)
    if not label in label_to_idx:
        label_to_idx[label] = len(idx_to_labels)
        idx_to_labels.append(label)
    labels.append(label_to_idx[label])
len(idx_to_labels)

In [ ]:
def fetch_pet(pet):
    img = image.load_img('pet_images/' + pet, target_size=(299, 299))
    return image.img_to_array(img)

img_vector = np.asarray([fetch_pet(pet) for pet in pet_images_fn])

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
for layer in base_model.layers:
    layer.trainable = False

pool_2d = GlobalAveragePooling2D(name='pool_2d')(base_model.output)
dense = Dense(1024, name='dense', activation='relu')(pool_2d)
predictions = Dense(len(idx_to_labels), activation='softmax')(dense)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# 0.9448
y = np.zeros((len(labels), len(idx_to_labels)))
for idx, label in enumerate(labels):
    y[idx][label] = 1

model.fit(
    img_vector, y,
    batch_size=128,
    epochs=15,
    verbose=2
)

In [ ]:
unfreeze = False
for layer in base_model.layers:
    if unfreeze:
        layer.trainable = True
    if layer.name == 'mixed9':
        unfreeze = True
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), 
              loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(
    img_vector, y,
    batch_size=128,
    epochs=10,
    verbose=2
)

In [ ]:
json.dump(idx_to_labels, open('../data/zoo/09.3 pet_labels.json', 'w'))
model.save('../data/zoo/09.3 retrained pet recognizer.h5')
